# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
O. Krause  ->  O. Krause  |  ['O. Krause']
T. Henning  ->  T. Henning  |  ['T. Henning']
O. Krause  ->  O. Krause  |  ['O. Krause']
F. Xu  ->  F. Xu  |  ['F. Xu']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 148 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.00410


extracting tarball to tmp_2509.00410...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.01069


extracting tarball to tmp_2509.01069...

...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2509.01937/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'mydefs' from 'tmp_2509.01937/mydefs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 61 bibliographic references in tmp_2509.01937/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2509.02128


/tmp/ipykernel_3377/2822249172.py:52: LatexWarning: 2509.02128 did not run properly
HTTP Error 404: Not Found
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01668-b31b1b.svg)](https://arxiv.org/abs/2509.01668) | **Azimuthal offsets in spiral arms of nearby galaxies**  |
|| M. Querejeta, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark>, <mark>K. Kreckel</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *21 pages, 8 figures. Accepted for publication in A&A*|
|**Abstract**|            Spiral arms play a central role in disc galaxies, but their dynamical nature remains a long-standing open question. Azimuthal offsets between molecular gas and star formation are expected if gas crosses spiral arms, as predicted by quasi-stationary density wave theory. In this work, we measure offsets between CO and Halpha peaks in radial bins for 24 galaxies from the PHANGS survey that display a well-delineated spiral structure. The offsets exhibit substantial scatter, implying that star formation is not exclusively initiated at a coherent spiral shock. We define offsets such that positive values mean Halpha peaks lie ahead of CO peaks in the direction of galactic rotation. With this convention, 14 galaxies show mean positive CO-Halpha offsets, typically of a few hundred parsecs. In four of these 14 galaxies (17% of the total), offsets become smaller with increasing radius, as expected for a single quasi-stationary spiral density wave. Ten galaxies (42%) show positive mean offsets but no clear correlation with radius, which is compatible with multiple overlapping modes. In the remaining ten galaxies (42%), we find no significantly positive offsets, which could point to transient dynamical spirals or material arms, where gas and stars co-rotate with the spiral perturbation. Across the full sample, we find mostly positive offsets between CO peaks and the gravitational potential minimum, confirming that gas often crosses the spiral perturbation. For the four galaxies with clear positive offsets and a radial trend, we derived pattern speeds in good agreement with the literature. Overall, our results suggest that even well-delineated spirals in the local Universe can arise from a variety of underlying dynamical mechanisms.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01800-b31b1b.svg)](https://arxiv.org/abs/2509.01800) | **The Far-Infrared Enhanced Survey Spectrometer (FIRESS) for PRIMA: Science Drivers**  |
|| K. M. Pontoppidan, et al. -- incl., <mark>T. Henning</mark>, <mark>O. Krause</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *Accepted for publication in JATIS*|
|**Abstract**|            We present the science drivers for the Far-Infrared Enhanced Survey Spectrometer (FIRESS), one of two science instrument on the PRobe Infrared Mission for Astrophysics (PRIMA). FIRESS is designed to meet science objectives in the areas of the origins of planetary atmospheres, the co-evolution of galaxies and supermassive black holes, and the buildup of heavy elements in the Universe. In addition to these drivers, FIRESS is envisioned as a versatile far-infrared spectrometer, capable of addressing science questions in most areas of astrophysics and planetary astronomy as part of a dominant General Observer (GO) program with 2/3 of the current science cases using FIRESS. We summarize how the instrument design choices and parameters enable the main science drivers as well as a broad and vibrant GO program.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01937-b31b1b.svg)](https://arxiv.org/abs/2509.01937) | **Radiative Transfer Modeling of a Shadowed Protoplanetary Disk assisted by a Neural Network**  |
|| J. P. Williams, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *accepted for publication in the Astrophysical Journal*|
|**Abstract**|            We present observations and detailed modeling of a protoplanetary disk around the T Tauri star, V1098 Sco. Millimeter wavelength data from the Atacama Large Millimeter Array (ALMA) show a ring of large dust grains with a central cavity that is filled with molecular gas. Near-infrared data with the Very Large Telescope (VLT) detect the scattered starlight from the disk surface and reveal a large shadow that extends over it's entire southern half. We model the ALMA continuum and line data to determine the outer disk geometry and the central stellar mass. Using radiative transfer models, we demonstrate that a misaligned inner disk, tilted in both inclination and position angle with respect to the outer disk, can reproduce the salient scattered light features seen with the VLT. Applying an image threshold algorithm to compare disk morphologies and training a neural network on a set of high signal-to-noise models, we forward model the data and determine the inner disk geometry. We find that the rotation axes of the inner and outer disks are misaligned by 38 degrees and constrain the mass and location of a perturbing planetary or substellar companion. The technique of simulation based inference that is illustrated here is broadly applicable for radiative transfer modeling of other objects.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.00410-b31b1b.svg)](https://arxiv.org/abs/2509.00410) | **Two-sided-loop jet originates from the filament internal reconnection**  |
|| Y. Huang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *15 pages, 5 figures, Accepted for Publication in RAA*|
|**Abstract**|            Magnetic reconnection driving two-sided-loop jet is typically associated with interactions between an emerging bipole and the overlying horizontal magnetic field, or between filaments from separate magnetic systems. Leveraging high temporal and spatial resolution observations from ground-based and space-borne instruments, we have identified a two-sided-loop jet originating from magnetic reconnection between threads within a single filament. Our observations show that as two initially crossing filamentary threads within the filament converge, reconnection takes place at their intersection. In the Doppler images, distinct redshift and blueshift signals are observed at the locations where the filament threads intersected. This process generates a two-sided-loop jet with outflow speeds of \speed{22.2} and \speed{62.5}. Following reconnection, the original crossing threads transform into two parallel threads that subsequently separate at speeds of \speed{2.8} and \speed{8.3}. This observation offers a new perspective on the mechanisms responsible for jet formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01069-b31b1b.svg)](https://arxiv.org/abs/2509.01069) | **A magnetic white dwarf formed through a binary merger within 35 million years**  |
|| H. Yan, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *16 pages, 9 figures, accepted for publication in ApJ Letters*|
|**Abstract**|            White dwarfs (WDs) represent the final evolutionary stage of most stars, typically originating from progenitor stars with masses below approximately 8 $M_{\odot}$ to 10 $M_{\odot}$. Formation through single-star evolution generally requires at least 25 Myr, with the youngest WDs often near the Chandrasekhar limit of 1.4 $M_{\odot}$. In contrast, WDs formed via binary channels, such as mergers or mass transfer, can develop smaller masses in a shorter timescale and may exhibit unique characteristics, including strong surface magnetic fields and rapid rotation. Accurately determining the ages of these WDs is essential for understanding their formation. A valuable method involves studying WDs in star clusters, where member stars share the same age and chemical composition, allowing for precise constraints on the formation times and metallicities of the WDs' progenitors. Here we report a WD found in the open cluster RSG 5, which is only 35 Myr old. The WD's mass is lower than 1.05 $M_{\odot}$, indicating it may not have formed through single-star evolution. The WD possesses an exceptionally strong surface magnetic field ($\ge 200$ MG), a short rotational period ($\sim 6.5$ min), and, most notably, a co-rotating half-ring of ionized circumstellar debris. This distinctive feature provides evidence for a binary merger origin, a scenario further substantiated by our stellar evolution models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01904-b31b1b.svg)](https://arxiv.org/abs/2509.01904) | **The ALMA-QUARKS survey: Extensive detection of acetamide in multiple high-mass star-forming regions**  |
|| C. Duan, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *10 pages, 8 figures, 3 tables, accepted for publication in A&A*|
|**Abstract**|            Acetamide (CH$_{3}$CONH$_{2}$), a key interstellar amide and a methyl derivative of formamide (NH$_{2}$CHO), has been sparsely detected, limiting insights into its prebiotic relevance. We present the first systematic survey for acetamide toward 52 hot molecular cores using ALMA Band 6 data. Acetamide has been detected in 10 cores, markedly expanding the inventory of known emitters. The derived column densities of acetamide range from $(2.5\pm0.9)\times10^{14}$ to $(1.5\pm0.6)\times10^{16}$ cm$^{-2}$, compared to formamide's $(1.1\pm0.1)\times10^{15}$ to $(6.9\pm0.4)\times10^{16}$ cm$^{-2}$. The nearly constant abundance ratios (~3-9) and strong abundance correlation between the two amides across sources suggest a chemically linked formation pathway, likely on grain surfaces. The presence of peptide-like molecules in these regions implies that complex organic species can survive star formation processes, offering a potential pathway toward prebiotic chemistry. These findings constrain the dominant grain surface formation routes of acetamide, confirm its broader prevalence in highmass star-forming regions, and underscore the importance of targeted amide surveys in tracing the chemical evolution toward prebiotic complexity.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 404: Not Found</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.02128-b31b1b.svg)](https://arxiv.org/abs/2509.02128) | **First JWST thermal phase curves of temperate terrestrial exoplanets reveal no thick atmosphere around TRAPPIST-1 b and c**  |
|| M. Gillon, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *72 pages, 4 main text Figures, 20 Extended Data Figures, 6 Supplementary Figures. Accepted for publication in Nature Astronomy*|
|**Abstract**|            We report JWST/MIRI 15 $\mu$m phase curves of TRAPPIST-1 b and c, revealing thermal emission consistent with their irradiation levels, assuming no efficient heat redistribution. We find that TRAPPIST-1 b shows a high dayside brightness temperature (490 $\pm$ 17 K), no significantly detectable nightside emission ($F_{\rm b, Night, max}$ = $39_{-27}^{+55}$ ppm), and no phase offset -- features consistent with a low-albedo, airless ultramafic rocky surface. TRAPPIST-1 c exhibits a lower dayside brightness temperature (369 $\pm$ 23 K), and a nightside flux statistically indistinguishable from that of TRAPPIST-1 b ($F_{\rm c, Night, max}$ = $62_{-43}^{+60}$ ppm). Atmosphere models with surface pressures $\geq$1 bar and efficient greenhouse effects are strongly disfavoured for both planets. TRAPPIST-1 b is unlikely to possess any substantial atmosphere, while TRAPPIST-1 c may retain a tenuous, greenhouse-poor O$_2$-dominated atmosphere or be similarly airless with a more reflective surface. These results suggest divergent evolutionary pathways or atmospheric loss processes, despite similar compositions. These measurements tightly constrain atmosphere retention in the inner TRAPPIST-1 system.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error HTTP Error 404: Not Found</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.01545-b31b1b.svg)](https://arxiv.org/abs/2509.01545) | **Complex spectral variability and hints of a luminous companion in the Be star + black hole binary candidate ALS 8814**  |
|| <mark>K. El-Badry</mark>, et al. -- incl., <mark>R. Seeburger</mark> |
|*Appeared on*| *2025-09-03*|
|*Comments*| *14 pages, 10 figures. Accepted to OJAp*|
|**Abstract**|            The emission-line binary ALS 8814 was recently proposed as a Be star + black hole (BH) binary based on large-amplitude radial velocity (RV) variations of a Be star and non-detection of spectroscopic features from a luminous companion. We reanalyze low- and medium-resolution LAMOST spectra of ALS 8814 and show that the system's spectroscopic variability is considerably more complex than previously recognized. Inspection of the system's trailed spectra reveals the presence of a second set of absorption lines that move in anti-phase with the Be star. In addition, the emission and absorption lines exhibit inconsistent RV variability, suggesting that they trace different stars. Using spectral disentangling, we recover the spectrum of a rapidly rotating companion whose broad, shallow lines were previously undetected. Time-variability in the emission lines complicates interpretation of the disentangled spectrum, such that the physical parameters of the components are still uncertain, but we find with simulations that emission line variability alone is unlikely to explain all signatures of the companion. The system's high Gaia RUWE value suggests a third luminous companion, making ALS 8814 a likely hierarchical triple. Although it is unlikely to contain a BH, the system is unusual, with the largest RV semi-amplitude observed in any known classical Be star and a companion that does not appear to be stripped. More extensive spectroscopic monitoring and high-resolution imaging will be necessary to fully characterize the system's orbital architecture, stellar parameters, and evolutionary status.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.01668/./Fig_classification.png', 'tmp_2509.01668/./Fig_methodology.png', 'tmp_2509.01668/./Fig_histogram.png']
copying  tmp_2509.01668/./Fig_classification.png to _build/html/
copying  tmp_2509.01668/./Fig_methodology.png to _build/html/
copying  tmp_2509.01668/./Fig_histogram.png to _build/html/
exported in  _build/html/2509.01668.md
    + _build/html/tmp_2509.01668/./Fig_classification.png
    + _build/html/tmp_2509.01668/./Fig_methodology.png
    + _build/html/tmp_2509.01668/./Fig_histogram.png
found figures ['tmp_2509.01800/./ppd_jatis.png', 'tmp_2509.01800/./galaxy_tracers.png', 'tmp_2509.01800/./OH_sim.png']
copying  tmp_2509.01800/./ppd_jatis.png to _build/html/
copying  tmp_2509.01800/./galaxy_tracers.png to _build/html/
copying  tmp_2509.01800/./OH_sim.png to _build/html/
exported in  _build/html/2509.01800.md
    + _build/html/tmp_2509.01800/./ppd_jatis.png
    + _build/html/tmp_2509.01800/./galaxy_tracers.png
    + _build/html/tmp_2509.01800/./OH_sim

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\re}{\ensuremath{r_\mathrm{e}}}$
$\newcommand{\SigStar}{\ensuremath{\Sigma_\mathrm{\star}}}$
$\newcommand{\SigMol}{\ensuremath{\Sigma_\mathrm{mol}}}$
$\newcommand{\SigSFR}{\ensuremath{\Sigma_\mathrm{SFR}}}$
$\newcommand{\tdep}{\ensuremath{\tau_\mathrm{dep}}}$
$\newcommand{\hi}{\ion{H}{I}}$
$\newcommand{\hii}{\ion{H}{II}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\Heidelberg}{\label{Heidelberg} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\COOL}{\label{COOL} Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\IRAP}{\label{IRAP} IRAP, OMP, Université de Toulouse, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\NRAOAb}{\label{NRAOAb} National Radio Astronomy Observatory, 800 Bradbury SE, Suite 235, Albuquerque, NM 87106, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, 4 Ivy Ln., Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UCSDAA}{\label{UCSDAA} Department of Astronomy \& Astrophysics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\UCT}{\label{UCT} Department of Astronomy, University of Cape Town, Rondebosch 7701, South Africa}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\esaSTScI}{\label{esaSTScI} AURA for the European Space Agency (ESA), ESA Office, Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} Institut Laue-Langevin, 71 avenue des Martyrs, F-38042 Grenoble, France}$
$\newcommand{\TUM}{\label{TUM} Technical University of Munich, School of Engineering and Design, Department of Aerospace and Geodesy, Chair of Remote Sensing Technology, Arcisstr. 21, 80333 Munich, Germany}$
$\newcommand{\Surrey}{\label{Surrey} Department of Physics, University of Surrey, Guildford GU2 7XH, UK}$
$\newcommand{\Oxford}{\label{Oxford} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\AIP}{\label{AIP} Leibniz-Institut for Astrophysik Potsdam (AIP), An der Sternwarte 16, 14482 Potsdam, Germany}$
$\newcommand{\StAndrews}{\label{StAndrews} School of Physics and Astronomy, University of St Andrews, North Haugh, St Andrews, KY16 9SS, UK}$
$\newcommand{\IAC}{\label{IAC}{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}}$
$\newcommand{\ULL}{\label{ULL}{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}}$
$\newcommand{\insubria}{ \label{insubria} Universit{à} dell’Insubria, via Valleggio 11, 22100 Como, Italy}$
$\newcommand{\Radcliffe}{\label{Radcliffe} Elizabeth S. and Richard M. Cashin Fellow at the Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\UniCA}{\label{UniCA} Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\Shizuoka}{\label{Shizuoka} Faculty of Global Interdisciplinary Science and Innovation, Shizuoka University, 836 Ohya, Suruga-ku, Shizuoka 422-8529, Japan}$
$\newcommand{\PMO}{\label{PMO}$
$Purple Mountain Observatory, No. 10 Yuanhua Road, Qixia District, 210023 Nanjing, China}$
$\newcommand{\Msun}{\ifmmode{\mathrm M_\odot}\else{M_\odot}\fi}$
$\newcommand\action{#1}$
$\newcommand\mod{#1}$
$\newcommand\mq{#1}$
$\newcommand\action{#1}$
$\newcommand{\mqb}{\vspace*{3mm} \noindent \bullet }$</div>



<div id="title">

# Azimuthal offsets in spiral arms of nearby galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.01668-b31b1b.svg)](https://arxiv.org/abs/2509.01668)<mark>Appeared on: 2025-09-03</mark> -  _21 pages, 8 figures. Accepted for publication in A&A_

</div>
<div id="authors">

M. Querejeta, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark>, <mark>K. Kreckel</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** Spiral arms play a central role in disc galaxies, but their dynamical nature remains a long-standing open question. Azimuthal offsets between molecular gas and star formation are expected if gas crosses spiral arms, as predicted by quasi-stationary density wave theory. In this work, we measure offsets between CO and H $\alpha$ peaks in radial bins for 24 galaxies from the PHANGS survey that display a well-delineated spiral structure. The offsets exhibit substantial scatter, implying that star formation is not exclusively initiated at a coherent spiral shock. We define offsets such that positive values mean H $\alpha$ peaks lie ahead of CO peaks in the direction of galactic rotation. With this convention, 14 galaxies show mean positive CO-H $\alpha$ offsets, typically of a few hundred parsecs. In four of these 14 galaxies (17 \% of the total), offsets become smaller with increasing radius, as expected for a single quasi-stationary spiral density wave. Ten galaxies (42 \% ) show positive mean offsets but no clear correlation with radius, which is compatible with multiple overlapping modes. In the remaining ten galaxies (42 \% ), we find no significantly positive offsets, which could point to transient dynamical spirals or material arms, where gas and stars co-rotate with the spiral perturbation. Across the full sample, we find mostly positive offsets between CO peaks and the gravitational potential minimum, confirming that gas often crosses the spiral perturbation. For the four galaxies with clear positive offsets and a radial trend, we derived pattern speeds in good agreement with the literature. Overall, our results suggest that even well-delineated spirals in the local Universe can arise from a variety of underlying dynamical mechanisms.

</div>

<div id="div_fig1">

<img src="tmp_2509.01668/./Fig_classification.png" alt="Fig6" width="100%"/>

**Figure 6. -** Classification of galaxies in our sample into three categories depending on whether they show positive mean CO-H$\alpha$ offsets or not and whether there is a significant trend with radius or not. \mod{The error bars are smaller than the size of the plotted circles.} The solid lines show running means for individual galaxies plotted against radius normalised by $R_{25}$. All galaxies are barred except for NGC 628, NGC 1385, NGC 2090, NGC 2997, and NGC 4254. (*fig:classification*)

</div>
<div id="div_fig2">

<img src="tmp_2509.01668/./Fig_methodology.png" alt="Fig3" width="100%"/>

**Figure 3. -** Top: Illustration showing the expectations for a non-exhaustive list of three scenarios that explain spiral structure: a quasi-stationary density wave (constant pattern speed), multiple overlapping modes (i.e. different pattern speeds across the disc), and material arms (always made up of the same stars, all of which are by definition co-rotating) or dynamical spirals (transient features made up of rather short, unconnected segments). Middle panels: Expected histograms for the distribution of CO-H$\alpha$ offsets ($\theta$) as well as the expected mean offset. We define offsets such that positive values mean H$\alpha$ peaks lie ahead of CO peaks in the direction of galactic rotation. For quasi-stationary density waves, we expect such offsets to be positive inside co-rotation and negative outside; however, since we sample mostly the inner parts of galaxies, the net offsets should typically be positive within our fields of view. The right panels indicate whether we should find a significant correlation between offsets and radius or not. Bottom: Illustration of how we measure azimuthal offsets in spiral arms in practice. For each radial bin (an elliptical annulus of width ${\sim}100$ pc that accounts for the inclination and PA of the galaxy), we average the CO and H$\alpha$ intensities in azimuthal bins. We then find the azimuth ($\phi$) corresponding to the maximum of both tracers within the footprint of each spiral mask ($\phi_{\rm max}$). The angular difference between the bins of peak intensity yields the offset of interest, $\theta$, at that radius. To account for galaxy rotation, if the galaxy rotates anti-clockwise, the offset $\theta$ is multiplied by $\eta = -1$.
 (*fig:method*)

</div>
<div id="div_fig3">

<img src="tmp_2509.01668/./Fig_histogram.png" alt="Fig5" width="100%"/>

**Figure 5. -** Histograms showing the distribution of the mean CO-H$\alpha$ and CO-NIR offsets across the whole galaxy sample. Each histogram includes one mean offset per galaxy, computed by averaging all measured offsets for all radii (top), the inner half (middle, $R < (R_{\rm min}^{\rm CO} + R_{\rm max}^{\rm CO})/2$), and the outer half (bottom, $R \ge (R_{\rm min}^{\rm CO} + R_{\rm max}^{\rm CO})/2$)).  (*fig:histo_CO-Ha*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.01668"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\apjl}{ApJL}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\nat}{Nature}$
$\newcommand{\baas}{BAAS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\araa}{ARA\&A}$
$\newcommand{\ssr}{Space Science Reviews}$
$\newcommand{\aapr}{The Astronomy \& Astrophysics Review}$
$\newcommand{\pasa}{PASA}$
$\newcommand{\cftdotsep}{\cftnodots}$</div>



<div id="title">

# The Far-Infrared Enhanced Survey Spectrometer (FIRESS) for PRIMA: Science Drivers $\footnote{\copyright 2025. All rights reserved. }$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.01800-b31b1b.svg)](https://arxiv.org/abs/2509.01800)<mark>Appeared on: 2025-09-03</mark> -  _Accepted for publication in JATIS_

</div>
<div id="authors">

K. M. Pontoppidan, et al. -- incl., <mark>T. Henning</mark>, <mark>O. Krause</mark>

</div>
<div id="abstract">

**Abstract:** We present the science drivers for the Far-Infrared Enhanced Survey Spectrometer (FIRESS), one of two science instrument on the PRobe Infrared Mission for Astrophysics (PRIMA). FIRESS is designed to meet science objectives in the areas of the origins of planetary atmospheres, the co-evolution of galaxies and supermassive black holes, and the buildup of heavy elements in the Universe. In addition to these drivers, FIRESS is envisioned as a versatile far-infrared spectrometer, capable of addressing science questions in most areas of astrophysics and planetary astronomy as part of a dominant General Observer (GO) program with 2/3 of the current science cases using FIRESS. We summarize how the instrument design choices and parameters enable the main science drivers as well as a broad and vibrant GO program.

</div>

<div id="div_fig1">

<img src="tmp_2509.01800/./ppd_jatis.png" alt="Fig1" width="100%"/>

**Figure 1. -** The bulk composition of planets reflect that of their natal protoplanetary disks at the disk radii where planets accrete their mass. For rocky planets, the bulk composition will be that of the solids of the disk, whereas giant planet atmospheres derive their composition from the disk gas. The total abundance of the disk ([C/H] and [O/H]) as well as the carbon-to-oxygen ratio (C/O) as measured by PRIMA-FIRESS are key tracers of how disk chemical evolution shapes planetary atmospheres. (*fig:disk_cartoon*)

</div>
<div id="div_fig2">

<img src="tmp_2509.01800/./galaxy_tracers.png" alt="Fig2" width="100%"/>

**Figure 2. -** FIRESS is capable of measuring the abundances of PAHs at redshifts $z=2-5$ and unobscured tracers of star formation and black hole accretion rates at $z>1$.  (*fig:galaxy_tracers*)

</div>
<div id="div_fig3">

<img src="tmp_2509.01800/./OH_sim.png" alt="Fig3" width="100%"/>

**Figure 3. -** Simulation of a FIRESS high-resolution observation of OH absorption from a galactic wind. In this case, the resolving power is tuned to $R=900$ to optimize the sensitivity to the line profile. (*fig:oh*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.01800"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\JW}[1]{{\color{blue} \bf JW; #1}}$
$\newcommand{\MV}[1]{{\color{green} \bf MV; #1}}$
$\newcommand{\MB}[1]{{\color{red} \bf MB; #1}}$
$\newcommand{\CG}[1]{{\color{orange} \bf CG; #1}}$
$\newcommand{\referee}[1]{{\bf #1}}$
$\newcommand{\kms}{\mbox{{\rm km~s^{-1}}}}$
$\newcommand{\Kkms}{\mbox{{\rm K km~s^{-1}}}}$
$\newcommand{ç}{\mbox{{\rm cm^{-3}}}}$
$\newcommand{\e}{\mbox{^{-1}}}$
$\newcommand{\ee}{\mbox{^{-2}}}$
$\newcommand{\eee}{\mbox{^{-3}}}$
$\newcommand{\cm2}{\mbox{{\rm cm^{-2}}}}$
$\newcommand{\h2}{\mbox{{\rm H}_2}}$
$\newcommand{\nh2}{\mbox{n_{\rm H_2}}}$
$\newcommand{\Nh2}{\mbox{N_{{\rm H}_2}}}$
$\newcommand{\Mh2}{\mbox{M_{{\rm H}_2}}}$
$\newcommand{\Msun}{\mbox{M_\odot}}$
$\newcommand{\Mearth}{\mbox{M_\oplus}}$
$\newcommand{\farcs}{\hbox{.\!\!^{"}}}$
$\newcommand{\farcm}{\hbox{.\!\!^{'}}}$
$\newcommand{\fs}{\hbox{.\!\!^{\rm s}}}$
$\newcommand{\micron}{\hbox{\mum}}$
$\newcommand{\simgt}{\lower.5ex\hbox{\; \buildrel > \over \sim \;}}$
$\newcommand{\simlt}{\lower.5ex\hbox{\; \buildrel < \over \sim \;}}$
$\newcommand{\pp}{\noindent\hangindent 20pt\hangafter=1}$
$\newcommand{\13CO}{^{13}CO}$
$\newcommand{\C18O}{C^{18}O}$
$\newcommand{\H2}{H_2}$
$\newcommand{\Mdust}{\mbox{M_{\rm dust}}}$
$\newcommand{\Mdisk}{\mbox{M_{\rm disk}}}$
$\newcommand{\Tdust}{\mbox{T_{\rm dust}}}$
$\newcommand{\Rtilt}{\mbox{R_{\rm tilt}}}$
$\newcommand{\itilt}{\mbox{i_{\rm tilt}}}$
$\newcommand{\PAtilt}{\mbox{{\rm PA}_{\rm tilt}}}$</div>



<div id="title">

# Radiative Transfer Modeling of a Shadowed Protoplanetary Disk assisted by a Neural Network

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.01937-b31b1b.svg)](https://arxiv.org/abs/2509.01937)<mark>Appeared on: 2025-09-03</mark> -  _accepted for publication in the Astrophysical Journal_

</div>
<div id="authors">

J. P. Williams, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** We present observations and detailed modeling of a protoplanetary disk around the T Tauri star, V1098 Sco. Millimeter wavelength data from the Atacama Large Millimeter Array (ALMA) show a ring of large dust grains with a central cavity that is filled with molecular gas. Near-infrared data with the Very Large Telescope (VLT) detect the scattered starlight from the disk surface and reveal a large shadow that extends over it's entire southern half. We model the ALMA continuum and line data to determine the outer disk geometry and the central stellar mass. Using radiative transfer models, we demonstrate that a misaligned inner disk, tilted in both inclination and position angle with respect to the outer disk, can reproduce the salient scattered light features seen with the VLT. Applying an image threshold algorithm to compare disk morphologies and training a neural network on a set of high signal-to-noise models, we forward model the data and determine the inner disk geometry. We find that the rotation axes of the inner and outer disks are misaligned by $38^\circ$ and constrain the mass and location of a perturbing planetary or substellar companion. The technique of simulation based inference that is illustrated here is broadly applicable for radiative transfer modeling of other objects.

</div>

<div id="div_fig1">

<img src="tmp_2509.01937/./representative_models.png" alt="Fig12" width="100%"/>

**Figure 12. -** A representative set of images showcasing the range of scattered light morphologies in the model grid. The parameter values for each model are shown in the top left corner of each panel. The central region is masked out to mimic the SPHERE coronagraph. Additional plots that show the systematic effects of changing a single parameter one at a time are shown in the Appendix.
 (*fig:representative_images*)

</div>
<div id="div_fig2">

<img src="tmp_2509.01937/./eddy_fit.png" alt="Fig3" width="100%"/>

**Figure 3. -** Model fit (left panel) and residuals (right panel) to the CO rotation map assuming a flat disk Keplerian profile.
 (*fig:eddy*)

</div>
<div id="div_fig3">

<img src="tmp_2509.01937/./mask_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Illustration of how the models are compared to the observations. The left panel shows the SPHERE image with borders defined by contouring to highlight the illuminated, northern side of the disk and the faint southern arc. A representative model is shown in the middle panel. An auto-thresholding algorithm then defines the regions of emission. The right panel shows binary images of the emission for the observations (red) and model (blue) which is used to quantify the goodness of fit.
 (*fig:mask*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.01937"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
